<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# # Batas

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
from bokeh.io import output_file, save
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, Select, CustomJS, HoverTool
from bokeh.layouts import column

# Buka file csvnya
hiv_data = pd.read_csv('hiv.csv')

# Tentukan nama file HTML untuk dijadikan
output_file('HIVisualbokeh.html')

# Membuat kolom datasource
source = ColumnDataSource(hiv_data)

# Membuat grafik dengan x merupakan jumlah kasus dan y merupakan kelompok umur yang angkanya disesuaikan dengan
# angka grafik
# Misalnya, y: -1 sampai 0 = Kelompok umur dari 0-4, 5-14 dan 15-19 tahun
# y: 0 smapai 1 = Kelompok umur dari 20-24, 25-49 dan 50 keatas.
p = figure(plot_width=400, plot_height=400, x_axis_label='Jumlah Kasus', y_axis_label='Kelompok Umur')

# Membuat hovertool untuk melihat keterangan Kota/Kabupaten di Jawa Barat
hover = HoverTool(tooltips=[('Kabupaten/Kota', '@nama_kabupaten_kota')])

# Membuat scatter plot untuk setiap gender.
gender_glyphs = {}
for gender in hiv_data['jenis_kelamin'].unique():
    gender_glyphs[gender] = p.circle('jumlah_kasus', 'kelompok_umur', source=source, size=8, color='red', alpha=0.5)

# Membuat selektor per jenis kelamin.
gender_list = list(hiv_data['jenis_kelamin'].unique())
select_gender = Select(title='Jenis Kelamin', options=gender_list, value='All')

# Membuat slider berdasarkan tahun.
slider = Slider(start=min(hiv_data['tahun']), end=max(hiv_data['tahun']), value=min(hiv_data['tahun']), step=1, title='Tahun')

# Membuat fungsi callback untuk mengatur slider dan selektornya
callback = CustomJS(args=dict(source=source, slider=slider, select_gender=select_gender, gender_glyphs=gender_glyphs), code="""
    const data = source.data;
    const value = slider.value;
    const selectedGender = select_gender.value;
    const x = data['jumlah_kasus'];
    const y = data['kelompok_umur'];
    const gender = data['jenis_kelamin'];
    const tahun = data['tahun'];

    for (let i = 0; i < x.length; i++) {
        if ((selectedGender === 'All' || gender[i] === selectedGender) && tahun[i] === value) {
            y[i] = Math.sin(x[i]);  // Modify the transformation according to your needs
        } else {
            y[i] = null;
        }
    }

    source.change.emit();

    for (const [gender, glyph] of Object.entries(gender_glyphs)) {
        if (selectedGender === 'All' || gender === selectedGender) {
            glyph.glyph.fill_alpha = 0.5;
        } else {
            glyph.glyph.fill_alpha = 0;
        }
    }
""")

# Memanggil fungsi callback agar bisa diubah dan niai pada plotnya juga berubah
select_gender.js_on_change('value', callback)
slider.js_on_change('value', callback)

# Membuat layout agar dapat ditampilkan
layout = column(select_gender, p, slider)

# Menyimpan file dalam bentuk .HTML
save(layout)

# Mengunduh HTML dikarenakan program dibuat di google colab
from google.colab import files
files.download('HIVisualbokeh.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>